<a href="https://colab.research.google.com/github/princetyagitech/NLP/blob/master/Email_Spam_not_spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%ls

drive/  sample_data/


In [3]:
% cd /content/drive/My Drive/Spam_Not_Spam

/content/drive/My Drive/Spam_Not_Spam


In [5]:
! git clone https://github.com/gognjanovski/SpamFilterMachineLearning

Cloning into 'SpamFilterMachineLearning'...
remote: Enumerating objects: 978, done.
remote: Total 978 (delta 0), reused 0 (delta 0), pack-reused 978
Receiving objects: 100% (978/978), 942.33 KiB | 882.00 KiB/s, done.
Resolving deltas: 100% (15/15), done.
Checking out files: 100% (971/971), done.


In [6]:
% cd /content/drive/My Drive/Spam_Not_Spam/SpamFilterMachineLearning/data

/content/drive/My Drive/Spam_Not_Spam/SpamFilterMachineLearning/data


# Loading Data

In [1]:
import os
import glob
import numpy as np
import email
import re
import string
from nltk.tokenize import word_tokenize

In [2]:
path='/content/drive/My Drive/Spam_Not_Spam/SpamFilterMachineLearning/data/'

In [3]:
nonspam_test= '/content/drive/My Drive/Spam_Not_Spam/SpamFilterMachineLearning/data/nonspam_test'
nonspam_train= '/content/drive/My Drive/Spam_Not_Spam/SpamFilterMachineLearning/data/nonspam_train'
spam_test= '/content/drive/My Drive/Spam_Not_Spam/SpamFilterMachineLearning/data/spam_test'
spam_train= '/content/drive/My Drive/Spam_Not_Spam/SpamFilterMachineLearning/data/spam_train'

In [4]:
spam_paths=[spam_test,spam_train]
nonspam_paths=[nonspam_test,nonspam_train]

In [5]:
def load_data():
  # loading spam_data
  data_spam_train=np.array([])
  data_spam_test=np.array([])
  for path_file in os.listdir(spam_train):
    tot_path=spam_train+'/'+path_file
    with open(tot_path) as f:
      a=f.readlines()
    data_spam_train=np.concatenate((data_spam_train,a),axis=0)

  for path_file in os.listdir(spam_test):
    tot_path=spam_test+'/'+path_file
    with open(tot_path) as f:
      a=f.readlines()
    data_spam_test=np.concatenate((data_spam_test,a),axis=0)

  # Loading Non_spam_data
  data_nonspam_train=np.array([])
  data_nonspam_test=np.array([])
  for path_file in os.listdir(nonspam_train):
    tot_path=nonspam_train+'/'+path_file
    with open(tot_path) as f:
      a=f.readlines()
    data_nonspam_train=np.concatenate((data_nonspam_train,a),axis=0)


  for path_file in os.listdir(nonspam_test):
    tot_path=nonspam_test+'/'+path_file
    with open(tot_path) as f:
      a=f.readlines()
    data_nonspam_test=np.concatenate((data_nonspam_test,a),axis=0)

  return data_spam_train,data_nonspam_train,data_spam_test,data_nonspam_test
  

In [6]:
data_spam_train,data_nonspam_train,data_spam_test,data_nonspam_test=load_data()

In [7]:
data_spam_train.shape,data_nonspam_train.shape,data_spam_test.shape,data_nonspam_test.shape

((350,), (350,), (130,), (130,))

In [8]:
nonspam_train_label = [0]*data_nonspam_train.shape[0]
spam_train_label = [1]*data_spam_train.shape[0]
x_train = np.concatenate((data_nonspam_train,data_spam_train))
y_train = np.concatenate((nonspam_train_label,spam_train_label))

In [9]:
nonspam_test_label = [0]*data_nonspam_test.shape[0]
spam_test_label = [1]*data_spam_test.shape[0]
x_test = np.concatenate((data_nonspam_test,data_spam_test))
y_test = np.concatenate((nonspam_test_label,spam_test_label))

In [10]:
train_shuffle_index = np.random.permutation(np.arange(0,x_train.shape[0]))
test_shuffle_index = np.random.permutation(np.arange(0,x_test.shape[0]))

In [11]:
x_train = x_train[train_shuffle_index]
y_train = y_train[train_shuffle_index]

In [ ]:
x_test = x_test[test_shuffle_index]
y_test = y_test[test_shuffle_index]

# Preprocessing Data
I am going to perform following preprocessing steps :

1. Lower words
2. Remove Numbers
3. Removing hyperlink
4. Replace_newline
5. Removing punctuations and whitespaces

In [12]:
def precoess_text(sentence):
  new_sent=re.sub(r"http\S+", "", sentence)
  new_sent = new_sent.lower()
  new_sent = re.sub(r'\d+', '', new_sent)
  new_sent = new_sent.translate(str.maketrans(dict.fromkeys(string.punctuation)))
  new_sent = new_sent.strip()
  new_sent=new_sent.replace('\n','')

  return new_sent

In [13]:
x_train = [precoess_text(sent) for sent in x_train]
x_test = [precoess_text(sent) for sent in x_test]

# Usig Stemmization,lemmitizer ,removing stop words

In [14]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [15]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [16]:
def next_preprocess(sentence):
  stemmer = PorterStemmer()
  lemmatizer = WordNetLemmatizer()
  # Removing Stop words
  new_sent = [i for i in sentence if i not in ENGLISH_STOP_WORDS]

  # Stemming
  new_sent=[stemmer.stem(word) for word in new_sent]

  #Lemmitizer
  new_sent=[lemmatizer.lemmatize(word) for word in new_sent]

  return new_sent

In [17]:
x_train = [next_preprocess(sent) for sent in x_train]
x_test = [next_preprocess(sent) for sent in x_test]

# Model

In [60]:

import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D,GRU
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [20]:
maxlen = 2000
max_features = 50000

In [21]:
EMBEDDING_FILE = '/content/drive/My Drive/Image_captioning/glove/glove.6B.200d.txt'
tokenizer = Tokenizer(num_words=max_features)

In [22]:
tokenizer.fit_on_texts(x_train)

In [23]:

x_train_features = np.array(tokenizer.texts_to_sequences(x_train))
x_test_features = np.array(tokenizer.texts_to_sequences(x_test))

x_train_features = pad_sequences(x_train_features,maxlen=maxlen)
x_test_features = pad_sequences(x_test_features,maxlen=maxlen)

In [24]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


In [50]:
!pip install tqdm
from tqdm.notebook import tqdm


In [53]:
def get_embedding_vectors(tokenizer, dim=200):
    
    embedding_index = {}
    with open(f"/content/drive/My Drive/Image_captioning/glove/glove.6B.200d.txt", encoding='utf8') as f:
        for line in tqdm(f, "Reading GloVe"):
            values = line.split()
            word = values[0]
            vectors = np.asarray(values[1:], dtype='float32')
            embedding_index[word] = vectors

    word_index = tokenizer.word_index
    embedding_matrix = np.zeros((len(word_index)+1, dim))
    for word, i in word_index.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            # words not found will be 0s
            embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

In [54]:
embedding_matrix = get_embedding_vectors(tokenizer)

In [58]:
EMBEDDING_SIZE=200
SEQUENCE_LENGTH = 100

In [72]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index)+1)

inp = Input(shape=(maxlen,))

x=Embedding(len(tokenizer.word_index)+1,
              EMBEDDING_SIZE,
              weights=[embedding_matrix],
              trainable=False,
              input_length=SEQUENCE_LENGTH)(inp)

x = GRU(64, recurrent_dropout=0.2,return_sequences=False)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.4)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 2000)]            0         
_________________________________________________________________
embedding_16 (Embedding)     (None, 2000, 200)         5200      
_________________________________________________________________
gru_5 (GRU)                  (None, 64)                51072     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 58,385
Trainable params: 53,185
Non-trainable params: 5,200
______________________________________________

In [73]:
model.layers[1].trainable = False

In [ ]:
history = model.fit(x_train_features,y_train, batch_size=512, epochs=20, 
          validation_data=(x_test_features, y_test))

Epoch 1/20
2/2 [==============================] - 6s 3s/step - loss: 0.6940 - accuracy: 0.5557 - val_loss: 0.6726 - val_accuracy: 0.6000
Epoch 2/20
2/2 [==============================] - 6s 3s/step - loss: 0.6778 - accuracy: 0.5800 - val_loss: 0.6648 - val_accuracy: 0.6269
Epoch 3/20
2/2 [==============================] - 6s 3s/step - loss: 0.6764 - accuracy: 0.5500 - val_loss: 0.6595 - val_accuracy: 0.5885
Epoch 4/20
2/2 [==============================] - 6s 3s/step - loss: 0.6607 - accuracy: 0.5900 - val_loss: 0.6526 - val_accuracy: 0.5769
Epoch 5/20
2/2 [==============================] - 6s 3s/step - loss: 0.6560 - accuracy: 0.6157 - val_loss: 0.6467 - val_accuracy: 0.6077
Epoch 6/20
2/2 [==============================] - 6s 3s/step - loss: 0.6537 - accuracy: 0.5929 - val_loss: 0.6433 - val_accuracy: 0.6115
Epoch 7/20
2/2 [==============================] - 6s 3s/step - loss: 0.6371 - accuracy: 0.6457 - val_loss: 0.6405 - val_accuracy: 0.6115
Epoch 8/20
2/2 [=========================

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

cnf_matrix = confusion_matrix(y_test,y_predict)

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Non Spam','Spam'], normalize=False,
                      title='Confusion matrix')

In [ ]:
def get_predictions(text):
    sequence = tokenizer.texts_to_sequences([text])
    # pad the sequence
    sequence = pad_sequences(sequence, maxlen=SEQUENCE_LENGTH)
    # get the prediction
    prediction = model.predict(sequence)[0]
    # one-hot encoded vector, revert using np.argmax
    return int2label[np.argmax(prediction)]

In [ ]:
text = "Congratulations! you have won 100,000$ this week, click here to claim fast"
print(get_predictions(text))